<a href="https://colab.research.google.com/github/Nekoiii/ML_Practices_colab/blob/main/image_classification_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
#connect to google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [27]:
#jump into the dataset folder
%cd '/content/gdrive/MyDrive/datasets/imgs/powerLines_classification'

/content/gdrive/MyDrive/datasets/imgs/powerLines_classification


In [28]:
import os
import cv2
import numpy as np
import glob as glob #glob module is used to search for files that match a specific file pattern or name. It can be used to search CSV files and for text in files. 
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

#Use the folder name as the class name.
path='./'
folders=os.listdir(path)
classes= [f for f in folders if (os.path.isdir(os.path.join(path,f)) and not f.startswith('.'))]
n_classes=len(classes)

print(classes,n_classes)

['with_powerlines', 'without_powerlines'] 2


In [29]:
import os
current_path = os.getcwd()
print(current_path)

/content/gdrive/MyDrive/datasets/imgs/powerLines_classification


In [ ]:
img=cv2.imread(image_path)
img=cv2.resize(img,dsize=(224,224))

#img

In [ ]:
X=[]
y=[]


lens=[]
for class_name in classes:
    lens.append( len(glob.glob(class_name + '/*.jpg')) )
min_samples = min(lens)


for label,class_name in enumerate(classes):
  files=glob.glob(class_name+'/*.jpg')
  #limit sample size (balance the number of samples for each classe)
  files = files[:min_samples] 

  for file in files: 
    #print(file)
    img=cv2.imread(file)
    if img is None:
      print('Failed to read the image: ',file)
    try: 
      img=cv2.resize(img,dsize=(224,224))
    except Exception as e:
      print("Error occurred while resizing the image:", e)
    
    X.append((img))
    y.append(label)
    
  print(X[0])
    

In [33]:
X=np.array(X)
X=X.astype('float32')
X /= 255
X[0]

array([[[0.99215686, 0.99215686, 0.98039216],
        [0.96862745, 0.9882353 , 0.99215686],
        [0.9529412 , 0.9882353 , 1.        ],
        ...,
        [0.98039216, 0.99215686, 0.99607843],
        [0.9882353 , 0.9843137 , 0.99215686],
        [0.9882353 , 0.9882353 , 0.9882353 ]],

       [[0.99215686, 0.99215686, 0.98039216],
        [0.96862745, 0.9882353 , 0.99215686],
        [0.9529412 , 0.9882353 , 1.        ],
        ...,
        [0.98039216, 0.99215686, 0.99607843],
        [0.9882353 , 0.9843137 , 0.99215686],
        [0.9882353 , 0.9882353 , 0.9882353 ]],

       [[0.99215686, 0.99215686, 0.98039216],
        [0.96862745, 0.9882353 , 0.99215686],
        [0.9529412 , 0.9882353 , 1.        ],
        ...,
        [0.98039216, 0.99215686, 0.99607843],
        [0.9882353 , 0.9843137 , 0.99215686],
        [0.9882353 , 0.9882353 , 0.9882353 ]],

       ...,

       [[1.        , 0.9882353 , 0.98039216],
        [0.9764706 , 0.9843137 , 0.99607843],
        [0.95686275, 0

In [34]:
unique_elements = set(y) 
counts = {element: y.count(element) for element in unique_elements}
print(counts)

{0: 368, 1: 368}


In [35]:
y-np.array(y)
y=np_utils.to_categorical(y,n_classes)
y[:5]

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [36]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(588, 224, 224, 3) (148, 224, 224, 3) (588, 2) (148, 2)


In [37]:
# VGG16: a convolution neural network (CNN) model supporting 16 layers
from keras.applications.vgg16 import VGG16  
# Sequential model: https://keras.io/guides/sequential_model/
# Sequential vs Model: https://stackoverflow.com/questions/66879748/what-is-the-difference-between-tf-keras-model-and-tf-keras-sequential
from keras.models import Sequential
from keras.models import Model
# model_from_json： loads a model structure from a JSON file or JSON string,and converts the JSON representation of a model into a Keras model object.
from keras.models import model_from_json
#Dropout: used for regularization and preventing overfitting in a neural network. It randomly sets a fraction of input units to 0 during training, which helps to prevent the network from relying too heavily on any single input feature.
from keras.layers import Input,Activation,Concatenate,Dense,Flatten,Dropout
from tensorflow.keras.optimizers import Adam


In [38]:
input_tensor= Input(shape=(224,224,3))
# include_top=False: exclude the top (fully connected) layers of the network from the model
base_model=VGG16(weights='imagenet',input_tensor=input_tensor,include_top=False)

#create the top layer 
top_model=Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(n_classes,activation='softmax'))

#combine base_model and top_model
model=Model(inputs=base_model.input,outputs=top_model(base_model.output))

#By freezing the weights of the first 15 layers, we can preserve the learned low-level features such as edge detection and texture that are generally applicable to many image classification tasks. 
#This prevents overfitting and speeds up training since these lower layers are already effective in capturing important visual patterns.
for layer in model.layers[:15]:
  layer.trainable=False

print('#layers_len=',len(model.layers))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

#print a useful summary of the model
model.summary()


#layers_len= 20
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56,

In [ ]:
model.fit(X_train,y_train,epochs=20,batch_size=16)

score=model.evaluate(X_test,y_test,batch_size=16)

In [ ]:
import pickle
pickle.dump(classes, open('./classes.sav', 'wb'))
model.save('./cnn_test_1.h5')


In [ ]:
from keras.models import load_model
import pickle
import cv2

model=load_model('./cnn_test_1.h5')
classes=pickle.load(open('./classes.sav','rb'))

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

PREDICT_MODE='SINGLE_IMG'  #'SINGLE_IMG','ALL_IMGS'

predict_folder='./predict_imgs/'

image_files=[]
if PREDICT_MODE=='SINGLE_IMG':
  image_files = glob.glob(predict_folder+'*.jpg') + glob.glob(predict_folder+'*.png')
elif PREDICT_MODE=='ALL_IMGS':
  image_files.append('./predict_imgs/test_8.jpg')

In [ ]:
predict_imgs=[]
for file_name in image_files:
  img=cv2.imread(file_name)
  img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
  #plt.imshow(img_rgb)
  #plt.axis('off') 
  #plt.show()
  if img is None:
    print('Can\'t find this file:',file_name)
    break
  img=cv2.resize(img,dsize=(224,224))
  img=img.astype('float32')
  img/=255.0
  predict_imgs.append(img)

predict_imgs=np.nstack(predict_imgs)
predict_results=model.predict(predict_imgs)


In [ ]:



output_df = pd.DataFrame(columns=['Image', 'Prediction Result', 'Predicted Class'])


# When there is only a single prediction image, we should add a batch dimension to the 'img' array.
#img=img[None,...]


for result, img_path in zip(predict_results, image_files):
  np.set_printoptions(precision=3,suppress=True)
  img = cv2.imread(img_path)
  pred = result.argmax()
  
  df = df.append({
      'Image': img,
      'Prediction Result': result * 100,
      'Predicted Class': classes[pred]
  }, ignore_index=True)


print(df)





